<a href="https://colab.research.google.com/github/pauloday/artbot/blob/main/Artbot_Studio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artbot Studio
by Paul O'Day

>Based on https://colab.research.google.com/drive/1L8oL-vLJXVcRzCFbPwOoMkPKJ8-aYdPN \

This is the runner for Artbot Studio, which will run in a different window. Here's how to run it:


1.   Go to the 'Runtime' menu, and select 'Change runtime type'. Make sure the accelerator is set to 'GPU'

2.   Run the init cell below by pressing the play button. You'll only have to run this once per session, even after you restart the runtime. It'll take a little while, it has to download at least 2GB of data to Google's servers, but the connection is really fast.

3.   Restart the runtime before running the next cell to get the studio link. Do this through the 'Runtime' menu, or by pressing ctrl-m then the period key.

4.   Run the next cell to get your URL

5.   Have fun!


In [ ]:
!git clone --quiet --branch pub https://github.com/pauloday/artbot
!mv artbot/* .
!./init.sh
!npm install -g localtunnel

     |████████████████████████████████| 7.9 MB 4.8 MB/s 
     |████████████████████████████████| 4.2 MB 59.3 MB/s 
     |████████████████████████████████| 111 kB 68.7 MB/s 
     |████████████████████████████████| 75 kB 4.1 MB/s 
     |████████████████████████████████| 170 kB 64.9 MB/s 
     |████████████████████████████████| 122 kB 71.8 MB/s 
     |████████████████████████████████| 786 kB 56.9 MB/s 
     |████████████████████████████████| 368 kB 59.6 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.19 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.0.3 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.26.0 which is incompatibl

**Restart the runtime before running the next cell**. If you don't, this cell will run and give you a URL but it won't work. Restarting the runtime won't destroy any data.

In [ ]:
!nohup streamlit run /content/artbot_studio.py &;
!lt --port 8501

Once you're done, stop the above cell (it may hang - just restart the runtime). Then run this cell to create a zip of all the images you made this session. You can download it (or move it to your Google drive if it's too big) at through the folder icon on the left menu bar. The zip will be called `Gaillery.zip`


In [ ]:
import shutil
shutil.make_archive('Gaillery', format='zip', root_dir='Gaillery')